<h1>Kata01: Supermarket Pricing</h1>


This kata arose from some discussions we’ve been having at the DFW Practioners meetings. The problem domain is something seemingly simple: pricing goods at supermarkets.

Some things in supermarkets have simple prices: this can of beans costs $0.65. Other things have more complex prices. For example:

    three for a dollar (so what’s the price if I buy 4, or 5?)
    $1.99/pound (so what does 4 ounces cost?)
    buy two, get one free (so does the third item have a price?)

This kata involves no coding. The exercise is to experiment with various models for representing money and prices that are flexible enough to deal with these (and other) pricing schemes, and at the same time are generally usable (at the checkout, for stock management, order entry, and so on). Spend time considering issues such as:

    does fractional money exist?
    when (if ever) does rounding take place?
    how do you keep an audit trail of pricing decisions (and do you need to)?
    are costs and prices the same class of thing?
    if a shelf of 100 cans is priced using “buy two, get one free”, how do you value the stock?

This is an ideal shower-time kata, but be careful. Some of the problems are more subtle than they first appear. I suggest that it might take a couple of weeks worth of showers to exhaust the main alternatives.
Goal

The goal of this kata is to practice a looser style of experimental modelling. Look for as many different ways of handling the issues as possible. Consider the various tradeoffs of each. What techniques are best for exploring these models? For recording them? How can you validate a model is reasonable?

In [19]:
import datetime

class SKU():
    """
    Stock Keeping Unit. The lowest level of stuff 
    """
    def __init__(self, description, manufacturer=None, material=None, 
                 size=None, color=None, packaging=None):
        self.description = description
        
        self.manufacturer = manufacturer        
        self.material = material
        self.size = size
        self.color = color
        self.packaging = packaging
        
    def __repr__(self):
        return "<%s {'description':%s, 'size': %s}>" % (type(self).__name__, self.description, self.size)

class Lot():
    """
    A box or pallet of SKUs
    timestamp is when they came into inventory
    """
    def __init__(self, SKU, quantity, unitCost, timestamp=None):
        self.SKU = SKU
        self.quantity = quantity
        self.unitCost = unitCost
        self.timestamp = timestamp
        
    def __repr__(self):
        return "<%s {'SKU':%s, 'quantity': %s, 'unitCost': %.2f, 'timestamp': %s }>" % (
            type(self).__name__, self.SKU, self.quantity, self.unitCost, self.timestamp.strftime("%Y-%m-%d"))
                
class UnorderedInventory():
    """
    A list of Lots where the in & out order doesn't matter
    """
    def __init__(self):
        self._inventory = [] # A list of Lots
        
    def __repr__(self):
        return "<%s {'_inventory':%s}>" % (type(self).__name__, self._inventory)
    
    def addLot(self, lot):
        self._inventory.append(lot)

    def addStock(self, SKU, quantity, unitCost, timestamp=None):
        if timestamp == None: 
            timestamp = datetime.datetime.now()
            
        self.addLot( Lot(SKU, quantity, unitCost, timestamp))

    def _find_lots(self, SKU):
        # For inherited classes, matching_lots should be sorted by stock rotation policy
        return [l for l in _inventory if l.SKU is SKU] 
            
    def drawStock(self, SKU, requested_quantity):
        """
        May return less than the requested_quantity if stock is not available
        """
        matching_lots = self._find_lots(SKU)
        r = 0

        while r < requested_quantity and len(matching_lots) > 0:
            diff = requested_quantity - r # 30 - 11
            # Transaction ??? Until then drawdown 1st then incr r
            if matching_lots[0].quantity >= diff:
                # This lot can meet the requirement                
                matching_lots[0].quantity -= diff
                r += diff
            else:
                # This lot can't meet the requirement                
                tmp = matching_lots[0].quantity
                del matching_lots[0]
                r += tmp
                
        return r
            
               
    def total_quantity(self, SKU):
        sum([l.quantity for l in _inventory if l.SKU is SKU])
        
    def total_unitCost(self, SKU):
        sum([l.unitCost for l in _inventory if l.SKU is SKU])
        

class LIFOInventory(UnorderedInventory): 
    """
    A list of Lots where Last In First Out applies
    """
    def _find_lots(self, SKU):
        # For inherited classes, matching_lots should be sorted by stock rotation policy
        return sorted([l for l in _inventory if l.SKU is SKU], key = lambda lot: lot.timestamp, reverse=True)
        
class FIFOInventory(UnorderedInventory): 
    """
    A list of Lots where First In First Out applies
    """
    def _find_lots(self, SKU):
        # For inherited classes, matching_lots should be sorted by stock rotation policy
        return sorted([l for l in _inventory if l.SKU is SKU], key = lambda lot: lot.timestamp, reverse=False)


class PricePoint():
    """
    The normal selling price of a SKU over a date range.
    """
    def __init__(self, SKU, price, fromDate, untilDate=None):
        self.SKU = SKU
        self.price = price
        self.fromDate = fromDate
        self.untilDate = untilDate  
        
    def __repr__(self):
        return "<%s {'SKU':%s, 'price': %.2f, 'fromDate': %s, 'untilDate': %s}>" % (
            type(self).__name__, 
            self.SKU, 
            self.price, 
            "None" if self.fromDate is None else self.fromDate.strftime("%Y-%m-%d"), 
            "None" if self.untilDate is None else self.untilDate.strftime("%Y-%m-%d"), 
        )
    
class PriceList():
    """
    A list of PricePoints (which have a relevant date range)
    """
    def __init__(self):
        self._prices = [] # A list of PricePoints
    
    def addPricePoint(self, pricePoint):
        # ToDo - deal with overlapping date ranges better than max()
        self._prices.append(pricePoint)
        
    def add(self, SKU, price, fromDate, untilDate=None):
        pp = PricePoint(SKU, price, fromDate, untilDate)
        self.addPricePoint(pp)
        
    def getPrice(self, SKU, as_at=None):
        """
        Returns the largest valid price.
        Throws a SKU Not Found exception.
        as_at defaults to now().
        """
        if as_at == None: 
            as_at = datetime.datetime.now()
        
        # Time for a lesson in list comprehension scopes :( 
        # Using lambda trick to get argument SKU into the list comprehension's scope
        # http://stackoverflow.com/a/28130950
        matchingPricePoints = (lambda SKU=SKU: [p for p in self._prices if p.SKU is SKU and 
                                    p.fromDate <= as_at and 
                                    (p.untilDate is None or p.untilDate < as_at)])()
        
        #if len(matchingPricePoints) <1:
        if matchingPricePoints is None:
            raise Exception('Price not found.')
            
        return sorted(matchingPricePoints, key=lambda pp: pp.price, reverse=True)[0].price
        

class DiscountFormula():
    """
    How much to knock off the price. 
    Factor is the percentage off & should normally be negative ( but could be positive for punitive pricing )
    """
    def __init__(self, description, factor=-0.0):
        self.description = description
        self.factor = factor
        
    def __str__(self):
        return self.description
        
    def value(self, quantity, unitPrice ):
        return quantity * unitPrice * self.factor
        

class DiscountFormula_m_for_n(DiscountFormula):
    """
    How much to knock off the price for a typical 3 for 2 special 
    Factor should normally be -1.0
    """
    def __init__(self, mfor, nfor, description, factor=-1.0):
        self.mfor = mfor
        self.nfor = nfor
        super().__init__(description, factor) 
        
    def __str__(self):
        return self.description
        
    def value(self, quantity, unitPrice):
        return int( (self.mfor - self.nfor) / self.mfor * quantity) * unitPrice * self.factor

class DiscountLine():
    """
    Line item on a PurchaseOrder
    """
    def __init__(self, SKU, quantity, unitPrice, discountFormula):
        self.SKU = SKU
        self.discountFormula = discountFormula
        self.value = discountFormula.value(quantity, unitPrice )
        
    def __str__(self):
        return "%s -> %s @ %.2f" % (self.discountFormula.description, self.SKU.description, self.value)
        
    
class OrderLine():
    """
    Line item on a PurchaseOrder
    """
    def __init__(self, SKU, quantity, unitPrice):
        self.SKU = SKU
        self.quantity = quantity
        self.unitPrice = unitPrice
        
    def __str__(self):
        return "%d * %s @ %.2f" % (self.quantity, self.SKU.description, self.unitPrice)
        
    def value(self):
        """Quantity * Price"""
        return (self.quantity * self.unitPrice)
    
    
class PurchaseOrder():
    """
    List of OrderLines and DiscountLines
    Specify a priceList to use when instantiating.
    """
    def __init__(self, priceList):
        self._priceList = priceList
        self.orderLines = []
        self.discountLines = []
        
    def __str__(self):
        return "\n".join([str(ol) for ol in self.orderLines])    + \
            "\nDiscounts \n"                                     + \
            "\n".join([str(dl) for dl in self.discountLines])    + \
            "\n------------------------------------------------" + \
            "\n               Total %.2f" % self.totalValue()    + \
            "\n================================================"
            
            
    def addOrderLine(self, SKU, quantity):
        ol = OrderLine(SKU, quantity, self._priceList.getPrice(SKU))
        self.orderLines.append(ol)
        
    def subtotalOrderLineValue(self, SKU):
        matchingOrderLines = (lambda SKU=SKU: [ol for ol in self.orderLines if ol.SKU is SKU])()
        return sum([ol.value() for ol in matchingOrderLines ])
    
    def subtotalOrderLineQuantity(self, SKU):
        matchingOrderLines = (lambda SKU=SKU: [ol for ol in self.orderLines if ol.SKU is SKU])()
        return sum([ol.quantity for ol in matchingOrderLines ])
            
    def addDiscountLine(self, SKU, discountFormula):
        dl = DiscountLine(SKU, self.subtotalOrderLineQuantity(SKU), self._priceList.getPrice(SKU), discountFormula)
        self.discountLines.append(dl)
       
    def totalValue(self):
        return sum([ol.value() for ol in self.orderLines]) + sum([dl.value for dl in self.discountLines])
    
    


<h2>ToDo</h2>
 * Send a purchaseOrder to Inventory for fulfillment
 * Draw down stock
 * Deal with insufficient stock 
 * Test drawdown ordering


<h2>Testing</h2>
 * Create some SKUs
 * Aquire some stock
 * Value that Stock
 * Create some specials
 * Create some orders
 * Value those orders with the discounts

In [20]:
baked_beans_sku    = SKU( description="Tin baked beans in tomato sauce", manufacturer="Heinz")
cannolli_beans_sku = SKU( description="Tin cannolli beans in brine", manufacturer="Koo")

warehouse1 = UnorderedInventory()
warehouse1.addStock(baked_beans_sku, 100, 2.99)
warehouse1.addStock(cannolli_beans_sku, 200, 1.99)

print(warehouse1)

<UnorderedInventory {'_inventory':[<Lot {'SKU':<SKU {'description':Tin baked beans in tomato sauce, 'size': None}>, 'quantity': 100, 'unitCost': 2.99, 'timestamp': 2016-07-06 }>, <Lot {'SKU':<SKU {'description':Tin cannolli beans in brine, 'size': None}>, 'quantity': 200, 'unitCost': 1.99, 'timestamp': 2016-07-06 }>]}>


In [21]:
retailPriceList = PriceList()
retailPriceList.add(baked_beans_sku, 3.99, datetime.datetime(1999, 1, 1), datetime.datetime(2009, 9, 9))
retailPriceList.add(baked_beans_sku, 3.99, datetime.datetime(2009, 9, 9))

retailPriceList.add(cannolli_beans_sku, 4.99, datetime.datetime(1988,  1,  1), datetime.datetime(2002, 2, 2))
retailPriceList.add(cannolli_beans_sku, 4.99, datetime.datetime(2002, 2, 1)) #Overlap!

po = PurchaseOrder(retailPriceList)
po.addOrderLine( baked_beans_sku, 5 )
po.addOrderLine( baked_beans_sku, 5 )

print(po)

5 * Tin baked beans in tomato sauce @ 3.99
5 * Tin baked beans in tomato sauce @ 3.99
Discounts 

------------------------------------------------
               Total 39.90


In [22]:
po.addOrderLine( cannolli_beans_sku, 6 )
po.addOrderLine( cannolli_beans_sku, 4 )

In [23]:
print(po)

5 * Tin baked beans in tomato sauce @ 3.99
5 * Tin baked beans in tomato sauce @ 3.99
6 * Tin cannolli beans in brine @ 4.99
4 * Tin cannolli beans in brine @ 4.99
Discounts 

------------------------------------------------
               Total 89.80


In [24]:
threeForTwoDiscountFormula = DiscountFormula_m_for_n( 3, 2, "Special 3 for 2",-1.0 )
pct80DiscountFormula = DiscountFormula( "Special Offer 15% off", -0.15)
po.addDiscountLine(baked_beans_sku, pct80DiscountFormula)
po.addDiscountLine(cannolli_beans_sku, threeForTwoDiscountFormula)

print(po)

5 * Tin baked beans in tomato sauce @ 3.99
5 * Tin baked beans in tomato sauce @ 3.99
6 * Tin cannolli beans in brine @ 4.99
4 * Tin cannolli beans in brine @ 4.99
Discounts 
Special Offer 15% off -> Tin baked beans in tomato sauce @ -5.99
Special 3 for 2 -> Tin cannolli beans in brine @ -14.97
------------------------------------------------
               Total 68.85


In [25]:
ol1 = (5*3.99)
ol2 = (5*3.99)
ol3 = (6*4.99)
ol4 = (4*4.99)

dl1 = (-0.15*10*3.99)
dl2 = (int((3-2)/3*10)*(-4.99))
print("%.2f, %.2f, %.2f, %.2f, %.2f, %.2f" % (ol1,ol2,ol3,ol4,dl1,dl2))
print("%.2f" % (ol1+ol2+ol3+ol4+dl1+dl2))


19.95, 19.95, 29.94, 19.96, -5.99, -14.97
68.85


In [26]:
print([ol.value() for ol in po.orderLines])
print([dl.value for dl in po.discountLines])


[19.950000000000003, 19.950000000000003, 29.94, 19.96]
[-5.985, -14.97]


In [ ]:
%debug
